# Homework on firm amenities

By Clara Kyung and Rebecca Wu* 

*We thank Conroy Lau, Feng Lin, and Xianglong Kong for helpful discussion. All errors are our own. 

*28 October, 2021*

## Preparing the environment

In [119]:
using Pkg
Pkg.activate(".") # Create new environment in this folder

Pkg.add(["Distributions","StatsBase"])
Pkg.add(["DataFrames","DataFramesMeta","Chain"])
Pkg.add(["Plots","Random","Missings"])
Pkg.add(["ShiftedArrays","CategoricalArrays"])
Pkg.add("NLsolve")
# Pkg.add(["SparseArrays","LightGraphs"])


# past the first time, you only need to instanciate the current folder
Pkg.instantiate() # Updates packages given .toml file

  Activating environment at `~/Documents/GitHub/ECON34430/PSET2/Project.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Project.toml`
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Project.toml`
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Project.toml`
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Project.toml`
  No Changes to `~/Documents/GitHub/ECON34430/PSET2/Manifest.toml`
   Resolving package versions...
   Installed Distances ─ v0.10.6
   Installed NLsolve ─── v4.5.1
    Updating `~/Documents/GitHub/ECON34430/PSET2/Project.toml`
  [2774e3e8] + NLsolve v4.5.1
    Updating `~/Documents/GitHub/ECON34430/PSET2/Manifest.toml`
  [

We then list our imports

In [300]:
using Distributions
using LinearAlgebra
using StatsBase
using DataFrames
using Plots
using CategoricalArrays
using ShiftedArrays
using Random
using Chain
using DataFramesMeta
using SparseArrays
using NLsolve
# using GLM

global_seed = 1240
Random.seed!(global_seed)

MersenneTwister(1240)

## Constructing Employer-Employee matched data

### Create a mobility matrix

One central piece is to have a network of workers and firms over time. We start by simulating such an object. The rest of the homework will focus on adding wages to this model. As we know from the lectures, a central issue of the network will be the number of movers.

We are going to model the mobility between workers and firms. Given a transition matrix we can solve for a stationary distribution, and then construct our panel from there.

In [301]:
# The mobility matrix will be the same. 
# But this time we draw a joint normal distribution of (ψ_j, v_j, and f_j). 

ψ_mean = 0 
v_mean = 0 
f_mean = 0 # need to be non-negative, this is fixed later

param_mean = [ψ_mean, v_mean, f_mean]

ψ_sd = 1
v_sd = 1
f_sd = 1
α_sd = 1

ψv_cov = 0.5
ψf_cov = 0.5 
vf_cov = 0.5 

param_cov = [ψ_sd ψv_cov ψf_cov; ψv_cov v_sd vf_cov; ψf_cov vf_cov f_sd]

nk = 30  # types of firms 
nl = 10  # types of workers

# Draw the firm parameters
firm_param_temp = rand(MvNormal(param_mean, param_cov), nk*10)
firm_param_temp = sortslices(firm_param_temp, dims = 2)
firm_param = zeros(3, nk)
for i in 1:nk
    firm_param[:, i] = mean(firm_param_temp[:,10*(i-1)+1: 10*i], dims = 2)
end
ψ = firm_param[1,:] # firm effect
α = quantile.(Normal(), (1:nl) / (nl + 1)) * ψ_sd # worker effect


csort = 0.5 # Sorting effect
# cnetw = 0.2 # We ignore network effect
csig  = 0.5 # Cross-sectional standard deviation

# Let's create type-specific transition matrices
# We are going to use joint normals centered on different values
G = zeros(nl, nk, nk)
for l in 1:nl, k in 1:nk
    # G[l, k, :] = pdf( Normal(0, csig), ψ .- cnetw * ψ[k] .- csort * α[l])
    G[l, k, :] = pdf( Normal(0, csig), ψ .- csort * α[l])
    G[l, k, :] = G[l, k, :] ./ sum(G[l, k, :])
end

# We then solve for the stationary distribution over psis for each alpha value
# We apply a crude fixed point approach
H = ones(nl, nk) ./ nk
for l in 1:nl
    M = transpose(G[l, :, :])
    for i in 1:100
        H[l, :] = M * H[l, :]
    end
end

### Simulate a panel

The next step is to simulate our network given our transition rules.

In [302]:
nt = 10
ni = 10000

# We simulate a balanced panel
ii = repeat(1:ni, 1, nt)  # Worker ID
ll = zeros(Int64, ni, nt) # Worker type
kk = zeros(Int64, ni, nt) # Firm type
spellcount = zeros(Int64, ni, nt) # Employment spell

# Specify rules for moving 
κ = 0.02 # death rate 
v = firm_param[2,:] # total firm value (wage + non-wage components)
exp_v = exp.(v)
λ = 0.168  # exogenous probability of receiving an offer, independent of firm and individual (number taken from Sorkin Table III)

# need to transform offer rate such that all values are positive and sum to 1 
f = exp.(firm_param[3,:])./ sum(exp.(firm_param[3,:])) # offer arrival rate 

# individual and time specific logit variable 
μ = rand(Logistic(), ni, nt)

for i in 1:ni
    
    # We draw the worker type
    l = rand(1:nl)
    ll[i,:] .= l
    
    # At time 1, we draw from H
    kk[i,1] = sample(1:nk, Weights(H[l, :]))
    
    for t in 2:nt
        if rand() < κ # dead 
            kk[i,t] = sample(1:nk, Weights(H[l, :])) # we re-draw from H (just as time 1)
            ii[i,t] = ni + ii[i,t-1] # replace the old worker id with a new id assigned to the new worker 
            ll[i,t] = rand(1:nl) # draw a new worker type
            spellcount[i,t] = 0 # reset spell
        else # alive 
            ii[i,t] = ii[i, t-1] # keep the previous worker id for workers who are alive
            
            if rand() < λ # receive an outside offer 
                kk[i, t] = sample(1:nk, Weights(f)) # draw the firm which the offer comes from
                
                if v[kk[i,t-1]] < v[kk[i,t]] + μ[i,t] # the new firm is better - move to new firm
                    spellcount[i,t] = spellcount[i,t-1] + 1
                else # the old firm is better - stay 
                    kk[i,t] = kk[i,t-1] # keep old firm id 
                    spellcount[i,t] = spellcount[i,t-1] 
                end 
                
            else # do not receive an outside offer 
                kk[i,t] = kk[i,t-1] # keep old firm id
                spellcount[i,t] = spellcount[i,t-1]
            end 
        end
    end 
    
end

### Attach firm ids to types

The final step is to assign identities to the firms. We are going to do this is a relatively simple way, by simply randomly assigning firm ids to spells.

In [303]:
firms_per_type = 15
jj = zeros(Int64, ni, nt) # Firm identifiers

draw_firm_from_type(k) = sample(1:firms_per_type) + (k - 1) * firms_per_type

for i in 1:ni
    
    # extract firm type
    k = kk[i,1]
    
    # We draw the firm (one of firms_per_type in given group)
    jj[i,1] = draw_firm_from_type(k)
    
    for t in 2:nt
        if spellcount[i,t] == spellcount[i,t-1] && ii[i,t] == ii[i, t-1] # we need both spell and worker id to be the same
            # We keep the firm the same
            jj[i,t] = jj[i,t-1]
        else
            # We draw a new firm
            k = kk[i,t]
            
            new_j = draw_firm_from_type(k)            
            # Make sure the new firm is actually new
            while new_j == jj[i,t-1]
                new_j = draw_firm_from_type(k)
            end
            
            jj[i,t] = new_j
        end
    end
end


# Make sure firm and worker ids are contiguous
contiguous_worker_ids = Dict( unique(ii) .=> 1:length(unique(ii))  )
ii .= getindex.(Ref(contiguous_worker_ids),ii);
contiguous_firm_ids = Dict( unique(jj) .=> 1:length(unique(jj))  )
jj .= getindex.(Ref(contiguous_firm_ids),jj);

In [304]:
# Create panel data with wages
tt = repeat((1:nt)',ni,1)
df = DataFrame(i=ii[:], j=jj[:], l=ll[:], k=kk[:], α=α[ll[:]], ψ=ψ[kk[:]], t=tt[:], exp_v = exp_v[kk[:]], f=f[kk[:]], spell=spellcount[:]);

# Add wages 
w_sigma = 0.2
df[!, :lw] = df.α + df.ψ + w_sigma * rand(Normal(), size(df)[1]);
sort!(df, [:i, :t, :k])


,i,j,l,k,α,ψ,t,exp_v,f,spell
,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Float64,Float64,Int64
1,1,1,3,8,-0.604585,-0.690609,1,0.601468,0.0226591,0
2,1,1,3,8,-0.604585,-0.690609,2,0.601468,0.0226591,0
3,1,89,3,6,-0.604585,-0.855096,3,0.939753,0.0114496,1
4,1,89,3,6,-0.604585,-0.855096,4,0.939753,0.0114496,1
5,1,89,3,6,-0.604585,-0.855096,5,0.939753,0.0114496,1
6,1,89,3,6,-0.604585,-0.855096,6,0.939753,0.0114496,1
7,1,89,3,6,-0.604585,-0.855096,7,0.939753,0.0114496,1
8,1,54,3,13,-0.604585,-0.227242,8,0.966957,0.0340834,2
9,1,54,3,13,-0.604585,-0.227242,9,0.966957,0.0340834,2


## Recover $V_j$ and $f_j$ in Sorkin's paper

### Step 1: identify $\alpha$ and $\psi$ using AKM

<!-- Estimation strategy: we want to recover $\alpha$, $\psi$, V, and f
1. identify $\alpha$ and $\psi$ using AKM. Here I decide not to pick the parameters to match the moment conditions for $\alpha$ and $\psi$ from Sokin's paper because we have different underlying DGP.  
2. somehow identify f? 
3. use equations 13 and 14 from Sorkin to identify V up to scale 
4. compute correlation between V and $\psi$  -->

In [305]:
# identify α and ψ using AKM 

function estimation(df::DataFrame, α_hat_init, ψ_hat_init; tol = 1e-9)
    df[!, :α_hat] .= α_hat_init
    df[!, :ψ_hat] .= ψ_hat_init
    mse = ones(1000);
    df_iter = df
    
    for i = 2:1000
        df_iter = @chain df_iter begin 

            @transform!(:α_temp = :lw - :ψ_hat)
            groupby(:i)
            transform!(:α_temp => mean => :α_hat) 

            @transform!(:ψ_temp = :lw - :α_hat)
            groupby(:j)
            transform!(:ψ_temp => mean => :ψ_hat) 
        end 
    
        mse[i] = mean((df_iter.lw - df_iter.α_hat - df_iter.ψ_hat).^2)
        if ((mse[i-1] - mse[i]) < tol)
            break
        end
    end 
    return df_iter
end

α_hat_init = 0.01
ψ_hat_init = 0.01

df_iter = estimation(df, α_hat_init, ψ_hat_init)


println("Correlation between true and estimated ψ: ", round(cor(df_iter.ψ, df_iter.ψ_hat), digits = 3))
println("Correlation between true and estimated α: ", round(cor(df_iter.α, df_iter.α_hat), digits = 3))

Correlation between true and estimated ψ: 0.999
Correlation between true and estimated α: 0.985


In [306]:
# # I try to follow Sean's approach here to create new firm type variable based on estimated ψ
# df_iter = @chain df_iter begin
#     groupby(:j)
#     combine(:j => unique => :j, :ψ_hat => mean => :ψ_hat)
#     @transform!(:k_new = levelcode.(cut(:ψ_hat, nk)))
#     select!(:j, :k_new)
#     innerjoin(df_iter, _ , on = :j)
# end
# DIDN'T CHANGE THE RESULT

### Step 2 : use worker inflow and outflow to identify $V_j$ and $f_j$ 

#### 2.1 : worker flows 

1. Conditional on receiving an offer from firm j', a worker from firm j decides to move if 
$Pr(V_j < V_{j'} + \mu) = \frac{e^{V_{j'}}}{e^{V_j} + e^{V_{j'}}} $ where $\mu$ is drawn from a standard logistic distribution. 

2. Outflow from firm j to firm j' can be represented as 

\begin{equation} 
\underbrace{M_{j'j}}_{\text{j to j' flows}} = \underbrace{g_j}_{\text{# of workers in j}} \underbrace{(1-\kappa)}_{\text{share of workers alive}} \underbrace{\lambda f_{j'} Pr(V_j < V_{j'} + \mu)}_{\text{get offer from j' and accept}}
\end{equation}

Sum over all firms other than j, we can get the total outflow from j: 

\begin{equation}
M_{Ej} = g_j (1-\kappa) \lambda \sum_{j'} f_{j'} Pr(V_j < V_{j'} + \mu)
\end{equation}

3. Inflow from firm j' to firm j can be represented as 

\begin{equation} 
\underbrace{M_{jj'}}_{\text{j' to j flows}} = \underbrace{g_j'}_{\text{# of workers in j'}} \underbrace{(1-\kappa)}_{\text{share of workers alive}} \underbrace{\lambda f_{j} Pr(V_{j'} < V_j + \mu)}_{\text{get offer from j and accept}}
\end{equation}

Sum over all firms other than j, we can get the total inflow to j: 

\begin{equation}
M_{jE} = f_j (1-\kappa) \lambda \sum_{j'} g_{j'} Pr(V_{j'} < V_j + \mu)
\end{equation}

#### 2.2 : iteration 

Taking $V_{j'}$ and $f_{j'}$ as given, we make an initial guess of $V_j$ and $f_j$. Then we use the total outflow equation to update $V_j$ and total inflow equation to update $f_j$. We iterate this step until the errors converge. 

In [307]:
# compute number of movers from type j to type j' (aggregate over time period b/c moves are independent of time)

sort!(df, [:i, :t])

df_flow = @chain df begin
    groupby(:i)
    transform(:j => lag => :j_lag, :k => lag => :k_lag)
    @transform!(:move = (:j .!= :j_lag) .* 1)
    transform(:move => (x -> ifelse.(ismissing.(x), 0, x)) => :move) # deal with missing values

    @subset(:move .== 1)
    groupby([:k, :k_lag])
    combine(:move => sum => :num_move)   
end   


# create inflow and outflow transition matrices for each FIRM TYPE (note that in the DGP, workers move between types, not particular firms.)

flow_mat = zeros(Int, nk, nk)
num_row = nrow(df_flow)

for i in 1:num_row 
    flow_mat[df_flow.k_lag[i], df_flow.k[i]] = df_flow.num_move[i]
end  

# sum over columns to get total number of inflow and outflow for each firm TYPE
M_Ej = sum(flow_mat, dims = 2) # total outflow
M_jE = sum(flow_mat, dims = 1)' # total inflow



30×1 adjoint(::Matrix{Int64}) with eltype Int64:
  27
  45
  59
  68
 115
  72
 108
  95
  86
 118
 167
 172
 221
   ⋮
 231
 238
 339
 230
 363
 281
 386
 379
 318
 739
 675
 784

In [308]:
# create g matrix - number of workers in each type of firms 
df_worker = @chain df begin 
    groupby(:k)
    combine(nrow => :num_worker)
    transform(:num_worker => (x -> x ./ sum(x)) => :share)
end

g_vec = zeros(nk, 1)
for i in 1:nk
    g_vec[df_worker.k[i], 1] = df_worker.share[i]
end

In [309]:
# create a matrix for initial guess of f_j and V_j  

f_j_init = ones(Int64, nk,1) ./ nk;
v_j_init = ones(Int64, nk,1);
x_init = [f_j_init; v_j_init];

function flow_soln(x)
    f_j = x[1:nk]; 
    v_j = x[nk+1:end];
    identity = sum(f_j) - 1;
    
    outflow_soln = M_Ej .- (1-κ) .* λ .* g_vec .* sum(f_j' .* exp.(v_j') ./ (exp.(v_j) .+ exp.(v_j')), dims = 2);
    inflow_soln = M_jE .- (1-κ) .* λ .* f_j .* sum(g_vec' .* exp.(v_j) ./ (exp.(v_j') .+ exp.(v_j)), dims = 1)';
    
    norm_M_Ej = M_Ej ./ M_Ej[1]; 
    norm_M_jE = M_jE ./ M_jE[1]; 
    RHS_outflow = g_vec .* sum(f_j' .* exp.(v_j') ./ (exp.(v_j) .+ exp.(v_j')), dims = 2);
    norm_RHS_outflow = RHS_outflow ./ RHS_outflow[1];
    
    RHS_inflow = f_j .* sum(g_vec' .* exp.(v_j) ./ (exp.(v_j') .+ exp.(v_j)), dims = 1)';
    norm_RHS_inflow = RHS_inflow ./ RHS_inflow[1]; 
    
    outflow_soln = norm_M_Ej[1:end-1] .- norm_RHS_outflow[1:end-1]; 
    inflow_soln = norm_M_jE .- norm_RHS_inflow;
    return[identity; outflow_soln; inflow_soln]
end

x_soln = nlsolve(flow_soln, x_init);
f_soln = x_soln.zero[1:nk];
V_soln = x_soln.zero[nk+1:end];


In [310]:
plot(1:nk, f_soln, label = "Estimated offer distribution")
plot!(1:nk, f, label = "Actual offer distribution")
cor(f, f_soln)
# cor(v, V_soln)

0.5676686863878878

In [33]:
# estimate death rate as the share of new workers entering the sample in each period 

death_rate = @chain df begin 
    groupby(:i)
    transform!(:t => minimum => :t_min)
    @transform(:new_worker = (:t_min .> 1).*(:t .== :t_min ))
    groupby(:t)
    combine(:new_worker => mean => :new_worker_mean)
    @subset(:t.!=1)
    mean(_.new_worker_mean)
end 
death_rate
println("Estimated death rate is ", round(death_rate, digits = 3))

Estimated death rate is 0.021
